In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.8-bin-hadoop2.7.tgz
!pip install findspark
!pip install pyspark
!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.8/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar"
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /content/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar pyspark-shell'

In [18]:
import findspark
findspark.init()

In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import Normalizer, StandardScaler
import random
import pyspark
import sys
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from uuid import uuid1
import time
import math
from pyspark.sql.functions import *
import pandas as pd

In [20]:
kafka_topic_name = "harry-potter-kvs"
kafka_bootstrap_servers = 'ec2-13-229-46-113.ap-southeast-1.compute.amazonaws.com:9092'

In [165]:
sc = pyspark.SparkContext()
ssc = StreamingContext(sc,20)

In [166]:
kvs = KafkaUtils.createStream(ssc, kafka_bootstrap_servers, 'spark-streaming-consumer', {kafka_topic_name:1}) 
kvs = KafkaUtils.createDirectStream(ssc, [kafka_topic_name], {"metadata.broker.list": kafka_bootstrap_servers})
kvs = KafkaUtils.createDirectStream(ssc, [kafka_topic_name], {
                        'bootstrap.servers':kafka_bootstrap_servers,
                        'group.id':'test-group',
                        'auto.offset.reset':'largest'})

In [158]:
w = kvs.window(20)
count = w.count()

# map1 = w.flatMap(lambda x: [((x[0],i),1) for i in x[1].split()])
# reduce = map1.reduceByKey(lambda x,y:x+y)
# tf = reduce.map(lambda x: (x[0][1],(x[0][0],x[1])))

count.pprint()

In [167]:
w = kvs.window(20)

count = w.count()

map1 = w.flatMap(lambda x: [((x[0],i),1) for i in x[1].split()])

reduce = map1.reduceByKey(lambda x,y:x+y)

tf = reduce.map(lambda x: (x[0][1],(x[0][0],x[1])))

map3 = reduce.map(lambda x: (x[0][1],(x[0][0],x[1],1)))

map4 = map3.map(lambda x:(x[0],x[1][2]))

reduce2 = map4.reduceByKey(lambda x,y:x+y)

idf=reduce2.map(lambda x: (x[0],math.log10(347/x[1])))

rdd=tf.join(idf)

rdd=rdd.map(lambda x: (x[1][0][0],(x[0],x[1][0][1],x[1][1],x[1][0][1]*x[1][1])))

rdd=rdd.map(lambda x: (x[0],x[1][0],x[1][1],x[1][2],x[1][3]))

count.pprint()
rdd.pprint()

In [168]:
ssc.start()

# stream will run for 120 sec
ssc.awaitTerminationOrTimeout(100)

-------------------------------------------
Time: 2021-09-24 15:55:00
-------------------------------------------
347

-------------------------------------------
Time: 2021-09-24 15:55:00
-------------------------------------------
('page5', '4', 1, 2.0632082200712114, 2.0632082200712114)
('page37', '4', 1, 2.0632082200712114, 2.0632082200712114)
('page42', '4', 1, 2.0632082200712114, 2.0632082200712114)
('page5', 'calls', 1, 2.2392994791268923, 2.2392994791268923)
('page44', 'calls', 1, 2.2392994791268923, 2.2392994791268923)
('page5', 'shouted', 1, 1.461148228743249, 1.461148228743249)
('page27', 'shouted', 1, 1.461148228743249, 1.461148228743249)
('page38', 'shouted', 1, 1.461148228743249, 1.461148228743249)
('page43', 'shouted', 1, 1.461148228743249, 1.461148228743249)
('page254', 'shouted', 1, 1.461148228743249, 1.461148228743249)
...

-------------------------------------------
Time: 2021-09-24 15:55:20
-------------------------------------------
0

-----------------------------

False

In [164]:
ssc.stop()
sc.stop()